<a href="https://colab.research.google.com/github/NadiaMusafarudin/NadiaMusafarudin/blob/main/Deep_Learning_Roberta_AZ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Text Classification/new_clean_data_spell.csv')
df = pd.read_csv('/content/drive/My Drive/thesis/dataset/new_clean_data_spell.csv') ## 0,1,2


In [ ]:
#df = df.drop('Unnamed: 0',axis = 1)

In [ ]:
df.shape

(10388, 3)

In [ ]:
df = df.dropna()

In [ ]:
df

,tweet,Label,Body
0,My nerdy nerd got vaccinated. Ã°Å¸Â¤Å“Ã°Å¸ÂÂ»...,1,By needy need got vaccinated. cucukmyaz jkjav ...
1,Ready for 2nd dose of vaccine tomorrow #CucukM...,1,Ready for nd dose of vaccine tomorrow CucukMyAZ
2,Done #CucukMyAZ #HartalDoktorKontrak https://...,1,One CucukMyAZ HartalDoktorKontrak
3,"Protection installed, FULLY vaccinated! Than...",1,Protection installed FULLY vaccinated got my n...
4,#CucukMyAZ 2nd dose done! https://t.co/QirsKs...,1,CucukMyAZ nd dose done
...,...,...,...
10383,So excited to get my vaccine #cucukMYAZ this ...,1,To excited to get my vaccine cucukMYAZ this co...
10384,ready 2 get my 1st jab #CucukMYAZ,1,ready get my st jaw CucukMYAZ
10385,I am ready to get the jab #cucukMYAZ on Friday...,1,I am ready to get the jaw cucukMYAZ on Friday ...
10386,Currently prepping for tomorrow's vaccination ...,1,Currently pressing for tomorrow is vaccination...


In [ ]:
df.isnull().sum()

tweet    0
Label    0
Body     0
dtype: int64

In [ ]:
df.dtypes

tweet    object
Label     int64
Body     object
dtype: object

In [ ]:
#df = df[['ActivityOn_Activity','result']]

In [ ]:
! pip install transformers

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 59 kB 6.7 MB/s 
     |████████████████████████████████| 895 kB 46.7 MB/s 
     |████████████████████████████████| 596 kB 47.9 MB/s 
     |████████████████████████████████| 3.3 MB 32.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import RobertaTokenizer, RobertaForSequenceClassification

from torch.utils.data import TensorDataset

#from transformers import BertTokenizer
#from torch.utils.data import TensorDataset

#from transformers import BertForSequenceClassification

#df = pd.read_csv('data/title_conference.csv')

#from transformers import RobertaConfig, RobertaModel

#>>> # Initializing a RoBERTa configuration
#>>> configuration = RobertaConfig()

#>>> # Initializing a model from the configuration
#>>> model = RobertaModel(configuration)

#>>> # Accessing the model configuration
#>>> configuration = model.config
df.head()

,tweet,Label,Body,label,data_type
0,My nerdy nerd got vaccinated. Ã°Å¸Â¤Å“Ã°Å¸ÂÂ»...,1,By needy need got vaccinated. cucukmyaz jkjav ...,0,train
1,Ready for 2nd dose of vaccine tomorrow #CucukM...,1,Ready for nd dose of vaccine tomorrow CucukMyAZ,0,train
2,Done #CucukMyAZ #HartalDoktorKontrak https://...,1,One CucukMyAZ HartalDoktorKontrak,0,train
3,"Protection installed, FULLY vaccinated! Than...",1,Protection installed FULLY vaccinated got my n...,0,val
4,#CucukMyAZ 2nd dose done! https://t.co/QirsKs...,1,CucukMyAZ nd dose done,0,val


In [ ]:
possible_labels = df.Label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{0: 1, 1: 0, 2: 2}

In [ ]:
df['label'] = df.Label.replace(label_dict)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.20, #original 0.15
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Label', 'label', 'data_type']).count()

tweet  Body
Label label data_type             
0     1     train       1115  1115
            val          279   279
1     0     train       6433  6433
            val         1609  1609
2     2     train        762   762
            val          190   190

In [ ]:
df['Label'].value_counts()


1    8042
0    1394
2     952
Name: Label, dtype: int64

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', 
                                          do_lower_case=True)
#tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Body.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Body.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
model = RobertaForSequenceClassification.from_pretrained("roberta-base",
                                    num_labels=len(label_dict),
                                    output_attentions=False,
                                    output_hidden_states=False)

#RobertaForSequenceClassification #RobertaModel

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=2e-5, 
                  eps=1e-8)
                  
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='macro')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda')

In [ ]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [ ]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

##training loop


In [ ]:
import random

seed_val = 17#42 #orignal 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):

    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_RoBERTA_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/2770 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
model = RobertaModel.from_pretrained("roberta-base",
                                    num_labels=len(label_dict),
                                    output_attentions=False,
                                    output_hidden_states=False)


model.to(device)

model.load_state_dict(torch.load('finetuned_RoBERTA_epoch_3.model', map_location=torch.device('cpu')))
#model.load_state_dict(torch.load('finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))


_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Class: 1
Accuracy: 1499/1609

Class: 0
Accuracy: 128/279

Class: 2
Accuracy: 107/190



In [ ]:
from sklearn.metrics import classification_report

preds_flat = np.argmax(predictions, axis=1).flatten()
labels_flat = true_vals.flatten()

print(classification_report(preds_flat, labels_flat))


              precision    recall  f1-score   support

           0       0.93      0.90      0.92      1659
           1       0.46      0.50      0.48       257
           2       0.56      0.66      0.61       162

    accuracy                           0.83      2078
   macro avg       0.65      0.69      0.67      2078
weighted avg       0.84      0.83      0.84      2078



In [ ]:
predictions

array([[ 4.1064467 , -0.93031764, -2.4737988 ],
       [ 4.437518  , -1.8482738 , -2.0218134 ],
       [ 4.024115  , -1.5901122 , -2.358473  ],
       ...,
       [ 4.191742  , -1.5191753 , -2.0654607 ],
       [ 0.60265356,  0.4974495 , -0.62150776],
       [ 4.4488444 , -1.5349389 , -2.297035  ]], dtype=float32)

In [ ]:
true_vals

array([0, 0, 0, ..., 0, 0, 0])